In [ ]:
import cv2
import dlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import time

# Define paths and parameters
IMG_WIDTH, IMG_HEIGHT = 224, 224
CLASSES = ['Closed_Eyes', 'Open_Eyes', 'No_yawn', 'Yawn']
MODEL_PATH = ''  # Path to your saved model that you wants to use
PREDICTOR_PATH = 'shape_predictor_68_face_landmarks.dat'  # dlib predictor
MOUTH_ROI_SCALE = 1.5  # Scale factor to enlarge mouth ROI
PADDING = 10  # Padding for eye ROIs to ensure sufficient size
VIDEO_FPS = 20  # Assumed FPS for fatigue metric calculations

# SEBlock function (from your model)
def se_block(input_tensor, reduction=8):
    filters = input_tensor.shape[-1]
    se = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
    se = tf.keras.layers.Reshape((1, 1, filters))(se)
    se = tf.keras.layers.Dense(filters // reduction, activation='relu', use_bias=False)(se)
    se = tf.keras.layers.Dense(filters, activation='sigmoid', use_bias=False)(se)
    x = tf.keras.layers.Multiply()([input_tensor, se])
    return x

# Load the trained model and recompile to suppress warning
model = tf.keras.models.load_model(MODEL_PATH, custom_objects={'se_block': se_block})
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Recompile to silence warning

# Preprocess image for model input
def preprocess_image(img):
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = img_to_array(img) / 255.0  # Normalize to [0, 1]
    return np.expand_dims(img, axis=0)

# Extract ROI using bounding box with padding
def extract_roi(image, landmarks, points, is_mouth=False):
    x_coords = [landmarks.part(i).x for i in points]
    y_coords = [landmarks.part(i).y for i in points]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    
    if is_mouth:
        # Scale the mouth ROI to make it larger
        width = x_max - x_min
        height = y_max - y_min
        x_min = max(0, x_min - int(width * (MOUTH_ROI_SCALE - 1) / 2))
        x_max = min(image.shape[1], x_max + int(width * (MOUTH_ROI_SCALE - 1) / 2))
        y_min = max(0, y_min - int(height * (MOUTH_ROI_SCALE - 1) / 2))
        y_max = min(image.shape[0], y_max + int(height * (MOUTH_ROI_SCALE - 1) / 2))
    else:
        # Add padding for eye ROIs to ensure sufficient size
        x_min = max(0, x_min - PADDING)
        x_max = min(image.shape[1], x_max + PADDING)
        y_min = max(0, y_min - PADDING)
        y_max = min(image.shape[0], y_max + PADDING)
    
    # Extract ROI
    roi = image[int(y_min):int(y_max), int(x_min):int(x_max)]
    if roi.size == 0:  # Handle empty ROI
        return None, None
    return roi, (int(x_min), int(y_min), int(x_max), int(y_max))

# Predict drowsiness state
def predict_drowsiness(roi, is_eye=True):
    if roi is None:
        return None, None
    processed_roi = preprocess_image(roi)
    prediction = model.predict(processed_roi, verbose=0)[0]
    if is_eye:
        pred_idx = np.argmax(prediction[:2])  # Only consider Closed_Eyes and Open_Eyes
        return pred_idx, prediction[pred_idx]
    else:
        pred_idx = np.argmax(prediction[2:]) + 2  # Only consider No_yawn and Yawn
        return pred_idx, prediction[pred_idx]

# Process live video feed with fatigue metrics
def process_live_video():
    # Load detector and predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(PREDICTOR_PATH)

    # Open webcam (default camera, usually index 0)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Failed to open webcam.")
        return

    # Initialize lists for tracking (based on 20 FPS)
    list_B = np.ones(int(VIDEO_FPS * 3))  # Eye status for 3s (open=1, closed=0)
    list_Y = np.zeros(int(VIDEO_FPS * 10))  # Mouth status for 10s (open=1, closed=0)
    list_Y1 = np.ones(int(VIDEO_FPS * 1.5))  # Yawn pattern (1.5s open then close)
    list_Y1[int(VIDEO_FPS * 1.5) - 1] = 0
    list_blink = np.ones(int(VIDEO_FPS * 10))  # Blink status for 10s (blink=1, no blink=0)
    list_yawn = np.zeros(int(VIDEO_FPS * 30))  # Yawn status for 30s (yawn=1, no yawn=0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame from webcam.")
            break

        # Prepare frame
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        flag_B = False  # Eye closed flag (True if any eye is closed)
        flag_Y = False  # Mouth open flag (True if mouth is Yawn)

        if len(faces) > 0:
            for face in faces:
                landmarks = predictor(gray, face)

                # Draw 68 landmarks on the frame
                for n in range(68):
                    x, y = landmarks.part(n).x, landmarks.part(n).y
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

                # Define landmark points
                LEFT_EYE_POINTS = list(range(36, 42))  # Left eye landmarks (36-41)
                RIGHT_EYE_POINTS = list(range(42, 48))  # Right eye landmarks (42-47)
                MOUTH_POINTS = list(range(48, 68))  # Mouth landmarks (48-67)

                # Extract ROIs
                left_eye, left_eye_coords = extract_roi(frame_rgb, landmarks, LEFT_EYE_POINTS)
                right_eye, right_eye_coords = extract_roi(frame_rgb, landmarks, RIGHT_EYE_POINTS)
                mouth, mouth_coords = extract_roi(frame_rgb, landmarks, MOUTH_POINTS, is_mouth=True)

                # Predict for eyes
                if left_eye is not None:
                    pred_idx, _ = predict_drowsiness(left_eye, is_eye=True)
                    if pred_idx is not None and pred_idx == 0:  # Closed_Eyes
                        flag_B = True
                if right_eye is not None:
                    pred_idx, _ = predict_drowsiness(right_eye, is_eye=True)
                    if pred_idx is not None and pred_idx == 0:  # Closed_Eyes
                        flag_B = True

                # Predict for mouth
                if mouth is not None:
                    pred_idx, _ = predict_drowsiness(mouth, is_eye=False)
                    if pred_idx is not None and pred_idx == 3:  # Yawn
                        flag_Y = True

                # Update lists
                start_time = time.time()
                if not flag_B:
                    list_B = np.append(list_B, 1)  # Open eyes
                else:
                    list_B = np.append(list_B, 0)  # Closed eyes
                list_B = np.delete(list_B, 0)

                if flag_Y:
                    list_Y = np.append(list_Y, 1)  # Open mouth
                else:
                    list_Y = np.append(list_Y, 0)  # Closed mouth
                list_Y = np.delete(list_Y, 0)

                # Detect blink (transition from open to closed)
                if len(list_B) >= 2 and list_B[-2] == 1 and list_B[-1] == 0:
                    list_blink = np.append(list_blink, 1)  # Blink detected
                else:
                    list_blink = np.append(list_blink, 0)  # No blink
                list_blink = np.delete(list_blink, 0)

                # Detect yawn (match list_Y1 pattern)
                if len(list_Y) >= len(list_Y1) and (list_Y[-len(list_Y1):] == list_Y1).all():
                    print('----------------------Yawn----------------------')
                    list_Y = np.zeros(int(VIDEO_FPS * 10))  # Reset mouth status after yawn
                    list_yawn = np.append(list_yawn, 1)  # Yawn detected
                else:
                    list_yawn = np.append(list_yawn, 0)  # No yawn
                list_yawn = np.delete(list_yawn, 0)

                # Calculate fatigue metrics
                perclos = 1 - np.average(list_B)  # Percentage of time eyes are closed
                perblink = np.average(list_blink)  # Average blink frequency
                peryawn = np.average(list_yawn)  # Average yawn frequency

                # Fatigue detection (thresholds from your code)
                if perclos > 0.4 or perblink < 2.5 / (10 * VIDEO_FPS) or peryawn > 3 / (30 * VIDEO_FPS):
                    fatigue_status = "Fatigued"
                else:
                    fatigue_status = "Alert"

                # Draw ROIs and text
                if left_eye_coords:
                    cv2.rectangle(frame, (left_eye_coords[0], left_eye_coords[1]),
                                 (left_eye_coords[2], left_eye_coords[3]), (255, 0, 0), 2)
                if right_eye_coords:
                    cv2.rectangle(frame, (right_eye_coords[0], right_eye_coords[1]),
                                 (right_eye_coords[2], right_eye_coords[3]), (255, 0, 0), 2)
                if mouth_coords:
                    cv2.rectangle(frame, (mouth_coords[0], mouth_coords[1]),
                                 (mouth_coords[2], mouth_coords[3]), (0, 0, 255), 2)

                # Display predictions and metrics
                left_eye_pred = predict_drowsiness(left_eye, is_eye=True)[0] if left_eye is not None else None
                right_eye_pred = predict_drowsiness(right_eye, is_eye=True)[0] if right_eye is not None else None
                mouth_pred = predict_drowsiness(mouth, is_eye=False)[0] if mouth is not None else None

                left_eye_text = f"Left Eye: {CLASSES[left_eye_pred] if left_eye_pred is not None else 'Not Detected'}"
                right_eye_text = f"Right Eye: {CLASSES[right_eye_pred] if right_eye_pred is not None else 'Not Detected'}"
                mouth_text = f"Mouth: {CLASSES[mouth_pred] if mouth_pred is not None else 'Not Detected'}"
                fps_text = f"fps: {1 / (time.time() - start_time):.2f}"
                fatigue_text = f"Status: {fatigue_status}"

                cv2.putText(frame, left_eye_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                cv2.putText(frame, right_eye_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                cv2.putText(frame, mouth_text, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                cv2.putText(frame, fps_text, (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                cv2.putText(frame, fatigue_text, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # Display the frame
        cv2.imshow('Drowsiness Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    process_live_video()